In [1]:
#%load_ext autoreload
#%autoreload 2
%matplotlib inline
%cd '../'

/Users/laurentperrinet/quantic/science/HomeHots/HOTS_clone_laurent


In [2]:

# https://laurentperrinet.github.io/sciblog/posts/2020-08-09-nesting-jupyter-runs.html
    
#verb =  (__name__ == "__main__")
def has_parent():
    """
    https://stackoverflow.com/questions/48067529/ipython-run-magic-n-switch-not-working
    
    Return True if this notebook is being run by calling
    %run in another notebook, False otherwise.
    """
    try:
        __file__
        # __file__ has been defined, so this notebook is 
        # being run in a parent notebook
        return True

    except NameError:
        # __file__ has not been defined, so this notebook is 
        # not being run in a parent notebook
        return False
def do_verb():
    return not has_parent()


do_it = do_verb()
if do_it : print('\ Running this notebook directly = ', do_it)    


\ Running this notebook directly =  True


In [3]:
import numpy as np
import matplotlib.pyplot as plt
import os
import time
# 
# https://en.wikipedia.org/wiki/ISO_8601
import datetime
timestr = datetime.datetime.now().date().isoformat()
timestr = '20201021'
timestr = '2020-11-03'
tau = 9e-4

from HOTS.Event import conv2eve
from HOTS.Tools import SaveObject, LoadObject
NbClusteringData = 2500
NbTrainingData = 5000
NbTestingData = 1500

In [4]:
def get_nmnist(timestr, NbTrainingData, NbTestingData, NbClusteringData, DataPath='Data/testsetnmnist.p'):
    fname_event_nmnist = f'Records/EXP_03_NMNIST/{timestr}_hots_event_nmnist.pkl'
    # print(help(LoadNMNIST))
    try:
        dataset = LoadObject(fname_event_nmnist)
        if verbose: print('loading the events from file', fname_event_nmnist)
    except:
        from HOTS.Event import LoadNMNIST
        dataset = LoadNMNIST(NbTrainingData, NbTestingData, NbClusteringData, 
                             Path=DataPath, verbose=0)
        SaveObject(dataset, fname_event_nmnist)
        if verbose: print('saving the events to file', fname_event_nmnist)

    events_train, events_test, event_cluster, labels_train, labels_test = dataset
    return events_train, events_test, event_cluster, labels_train, labels_test


In [5]:
def get_events(timestr, NbTrainingData, NbTestingData, NbClusteringData, 
               tau=tau, # -> tau=1ms, si on prend 10 ms on est à 1s pour la dernière couche et les vidéos font 0.3s en moyenne
               homeo=True, verbose=False):
    R = 2    
    filthr = 2
    nbkNN = 3
    algo = 'lagorce'
    decay = 'exponential'
    krnlinit = 'rdn'
    nb_cluster = [4, 8, 16]

    if homeo:
        fname_model = 'Records/EXP_03_NMNIST/'+timestr+'_hots_'+str(tau*1000)+'ms_'+algo+'_homeo.pkl'
        fname_event0_o = 'Records/EXP_03_NMNIST/'+timestr+'_hots_'+str(tau*1000)+'_event0_o_homeo.pkl'
    else:
        fname_model = 'Records/EXP_03_NMNIST/'+timestr+'_hots_'+str(tau*1000)+'ms_'+algo+'.pkl'
        fname_event0_o = 'Records/EXP_03_NMNIST/'+timestr+'_hots_'+str(tau*1000)+'_event0_o.pkl'

    if not os.path.isfile(fname_event0_o):            
        if verbose: print('creating the events in file', fname_event0_o)
        from HOTS.ToolsMonitor import GenerateActivationMap, DisplayActivationMap
        from HOTS.Event import Event, SimpleAlphabet, LoadNMNIST
        from HOTS.STS import STS
        from HOTS.Layer import ClusteringLayer
        from HOTS.ToolsMonitor import (
            DisplayImage,
            DisplaySurface3D,
            DisplaySurface2D,
            DisplayConvergence,
        )
        from HOTS.Network import Network
        from HOTS.Event import conv2eve

        events_train, events_test, event_cluster, labels_train, labels_test = get_nmnist(timestr, NbTrainingData, NbTestingData, NbClusteringData)
        
        L1 = ClusteringLayer(tau=tau,R=R,verbose=0,ThrFilter=filthr,LearningAlgo=algo,kernel=decay,homeo=homeo,init=krnlinit)
        L2 = ClusteringLayer(tau=10 * tau,R=2 * R,verbose=0,ThrFilter=filthr,LearningAlgo=algo,kernel=decay,homeo=homeo,init=krnlinit)
        L3 = ClusteringLayer(tau=10 * 10 * tau,R=2 * 2 * R,verbose=0,ThrFilter=filthr,LearningAlgo=algo,kernel=decay,homeo=homeo,init=krnlinit)
        Net = Network([L1, L2, L3])    


        if not os.path.isfile(fname_model):
            ClusterLayer, event_output = Net.TrainCluster(
                    event=event_cluster, NbClusterList=nb_cluster, to_record=True, NbCycle=1
                )
            SaveObject(ClusterLayer, fname_model)
        else: 
            if verbose: print('loading model from file', fname_model)

            ClusterLayer, Classif0 = LoadObject(fname_model)

        if verbose: print('run the events through the network')
        homrun = False
        events_train_o = Net.RunNetwork(events_train, NbClusterList=ClusterLayer, homrun=homrun)
        events_test_o = Net.RunNetwork(events_test, NbClusterList=ClusterLayer, homrun=homrun)
        SaveObject([events_train, events_test], fname_event0_o)
    else: 
        if verbose: print('loading the events from file', fname_event0_o)
        events_train_o, events_test_o = LoadObject(fname_event0_o)
        
    return events_train_o, events_test_o


### Building matrix for logistic regression

In [6]:
def gather_data(events_in, labels, 
                tau_cla=.150, # characteristic time of a digit 
                sample_events=200, sample_space = 1,
                verbose=False):
    c_int = lambda n, d : ((n - 1) // d) + 1

    n_events = events_in.time.shape[0]

    data = np.zeros((c_int(events_in.ImageSize[0], sample_space), 
                     c_int(events_in.ImageSize[1], sample_space), 
                     len(events_in.ListPolarities))) #tmp data

    X = np.zeros((c_int(n_events, sample_events), len(data.ravel())))
    y = np.zeros((c_int(n_events, sample_events), ))

    t_absolute = 0
    t_old = 0
    i_labels = 0
    for i in range(1, n_events):
        if i == events_in.ChangeIdx[i_labels]:
            i_labels += 1
            t_absolute += events_in.time[i]
        t_new = t_absolute + events_in.time[i]
        
        data *= np.exp(-(t_new-t_old)/tau_cla)
        
        t_old = t_new

        data[events_in.address[i, 0]//sample_space, 
             events_in.address[i, 1]//sample_space, 
             events_in.polarity[i]] = 1.

        if i % sample_events == 0:
            #if verbose: print(f'{i=} {n_events=} {n_events//sample_events=}')
            X[i//sample_events, :] = data.ravel()
            y[i//sample_events] = labels[i_labels]

    if verbose: print('Number of events: ' + str(X.shape[0])+' - Number of features: ' + str(X.shape[1]))

    return X, y


In [7]:
from sklearn.linear_model import LogisticRegression as LR
# from sklearn.linear_model import LogisticRegressionCV as LR
from sklearn.model_selection import train_test_split

opts_LR = dict(random_state=0, max_iter=5000, n_jobs=-1, class_weight='balanced')
#opts_LR['Cs'] = 5
#opts_LR['Cs'] = 32


In [8]:
if do_it : 
    tic = time.time()
    dataset = get_nmnist(timestr, NbTrainingData, NbTestingData, NbClusteringData, verbose=True)
    events_train, event_test, event_cluster, labels_train, labels_test = dataset
    print(f'Done in {time.time() - tic:.3f}')


Done in 124.789


In [9]:
events_train.ListPolarities
events_train.time

array([0.00068 , 0.003483, 0.008972, ..., 0.302477, 0.305744, 0.308378])

In [ ]:
if do_it : 
    opts_LR = dict(random_state=0, max_iter=500, n_jobs=-1, class_weight='balanced')

    tic = time.time()
    X_train, y_train = gather_data(events_train, labels_train, verbose=True)
    print(f'Done in {time.time() - tic:.3f}')

    tic = time.time()
    lr = LR(**opts_LR).fit(X_train, y_train)
    print(f'Done in {time.time() - tic:.3f}')
    print(f'Classification score for raw input is {lr.score(X_train, y_train):.3f}')

    tic = time.time()
    X_test, y_test = gather_data(events_test, labels_test, verbose=True)
    print(f'Done in {time.time() - tic:.3f}')

    print(f'Classification score for raw input is {lr.score(X_test, y_test):.3f}')

Number of events: 104930 - Number of features: 2312
Done in 111.195


In [ ]:
if do_it : 
    tic = time.time()
    events_train_o, events_test_o = get_events(timestr, tau=tau, homeo=True, verbose=True)
    print(f'Done in {time.time() - tic:.3f}')

    tic = time.time()
    X_train, y_train = gather_data(events_train, labels_train, verbose=True)
    print(f'Done in {time.time() - tic:.3f}')

    tic = time.time()
    lr = LR(**opts_LR).fit(X_train, y_train)
    print(f'Classification score for raw input is {lr.score(X_train, y_train):.3f}')
    print(f'Done in {time.time() - tic:.3f}')

    tic = time.time()
    X_test, y_test = gather_data(events_test, labels_test, verbose=True)
    print(f'Done in {time.time() - tic:.3f}')
    print(f'Classification score for raw input is {lr.score(X_test, y_test):.3f}')


In [ ]:
if do_it : 
    tic = time.time()
    dataset = get_nmnist(timestr, NbTrainingData, NbTestingData, NbClusteringData)
    events_train, event_test, event_cluster, labels_train, labels_test = dataset
    print(f'Done in {time.time() - tic:.3f}')


In [ ]:
if do_it : 
    opts_LR = dict(random_state=0, max_iter=500, n_jobs=-1, class_weight='balanced')

    tic = time.time()
    X_train, y_train = gather_data(events_train, labels_train, verbose=True)
    print(f'Done in {time.time() - tic:.3f}')

    tic = time.time()
    lr = LR(**opts_LR).fit(X_train, y_train)
    print(f'Done in {time.time() - tic:.3f}')
    print(f'Classification score for raw input is {lr.score(X_train, y_train):.3f}')

    tic = time.time()
    X_test, y_test = gather_data(events_test, labels_test, verbose=True)
    print(f'Done in {time.time() - tic:.3f}')

    print(f'Classification score for raw input is {lr.score(X_test, y_test):.3f}')

In [ ]:
if do_it : 
    tic = time.time()
    events_train_o, events_test_o = get_events(timestr, tau=tau, homeo=True, verbose=True)
    print(f'Done in {time.time() - tic:.3f}')

    tic = time.time()
    X_train, y_train = gather_data(events_train, labels_train, verbose=True)
    print(f'Done in {time.time() - tic:.3f}')

    tic = time.time()
    lr = LR(**opts_LR).fit(X_train, y_train)
    print(f'Classification score for raw input is {lr.score(X_train, y_train):.3f}')
    print(f'Done in {time.time() - tic:.3f}')

    tic = time.time()
    X_test, y_test = gather_data(events_test, labels_test, verbose=True)
    print(f'Done in {time.time() - tic:.3f}')
    print(f'Classification score for raw input is {lr.score(X_test, y_test):.3f}')
